In [7]:
import race_results_scanner
import google_cloud_downloader as gcd
from PyPDF2 import PdfReader
import numpy as np
import pandas as pd
import os
import header_scanner as hs


pdf = "Error_PDFS/ARP_10_01_2022.pdf"


In [2]:
#Testing for page

page_list = race_results_scanner.get_page_list(pdf)
target_page = 7

page_list_index = 0
for page in page_list:
    if(target_page == page['page_num']):
        break
    page_list_index += 1
    
reader = PdfReader(pdf) #File to be scanned
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()

page = page_list[page_list_index]
header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
for field,value in header.items():
    print(field ,':', value)
result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan
print(result_tables[0])
print(result_tables[1])

The PDF <_io.BufferedReader name='Error_PDFS/ARP_09_18_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


track : Arapahoe Park
date : September 18, 2022
race_num : 8
criteria : THREE YEAR OLDS REMAINING ELIGIBLE TO THE TRIALS
track_length :  Four Hundred Yards On The Dirt
purse : $6,900
weather : Clear
track_type : Fast
off_time : 3:56
start : Good for all
race_type : derby trail
Looking for tables...
Found table top bound at 124.7299999999999
Found table bottom bound at 205.13
Looking for tables...
Found table top bound at 352.93
Found table bottom bound at 444.7080000000001
    Last Raced  Pgm                        Horse Name (Jockey)  Start Str 1  \
0  28Aug22 ARP  1.0               Mr Dash Fire (Flores, James)    1.0     1   
1  14Aug22 ARP  5.0        Flare for Royalty (Lazalde, Martin)    2.0     2   
2  24Jul22 ARP  8.0  Yesterday Road (Aguirre-Erives, Cristian)    7.0     5   
3  22Aug22 ARP  3.0      Paint Your Face Blue (Garcia, Ramiro)    4.0     3   
4   5Aug22 PRM  2.0         Carisma Vista (Montalvo, Genovero)    3.0     4   
5  30Aug22 ARP  7.0                  Ms Eireli (

In [3]:
result_tables[0]

,Last Raced,Pgm,Horse Name (Jockey),Start,Str 1,Str,Fin,Odds,Ind. Time,Sp. In.,Comments,Last Raced_super_script,Str 1_length_behind,Str_length_behind,Fin_length_behind,Wgt,M/E
0,28Aug22 ARP,1.0,"Mr Dash Fire (Flores, James)",1.0,1,1,1,0.90*,20.411,86.0,broke best edg,8 2,Neck,1/2,1/2,124,BL b
1,14Aug22 ARP,5.0,"Flare for Royalty (Lazalde, Martin)",2.0,2,2,2,8.30,20.482,85.0,away well outfi,7 4,Neck,1/2,1/2,124,BL bf
2,24Jul22 ARP,8.0,"Yesterday Road (Aguirre-Erives, Cristian)",7.0,5,5,3,2.90,20.569,82.0,broke out shar,8 4,1 1/2,2,Head,124,BL bf
3,22Aug22 ARP,3.0,"Paint Your Face Blue (Garcia, Ramiro)",4.0,3,3,4,9.40,20.583,82.0,away well caug,2 1,Neck,Head,Neck,124,B bf
4,5Aug22 PRM,2.0,"Carisma Vista (Montalvo, Genovero)",3.0,4,4,5,5.30,20.633,81.0,away well outfi,1 3,Neck,Head,2 1/2,124,BL b
5,30Aug22 ARP,7.0,"Ms Eireli (Soto, Jhaciel)",6.0,6,6,6,32.90,21.045,71.0,always well ba,1 6,1/2,1/2,3/4,124,BL bfk
6,23Aug22 ARP,6.0,"Llego El Amo (Valles, Daniel)",5.0,7,7,7,11.40,21.166,68.0,outrun no rally,1 4,NaN,NaN,NaN,124,BL bf


In [5]:
df = result_tables[0]
col_list = list(df.columns.values)
for col_name in col_list:
        if(col_name.__contains__("Unnamed")):
            df = df.drop(col_name,axis=1)
df

,Last Raced,Pgm,Horse Name (Jockey),Wgt M/E PP,Start,Str 1,Str,Fin,Odds,Ind. Time,Sp. In.,Comments
0,8 2,NaN,NaN,NaN,NaN,Neck,1/2,1/2,NaN,NaN,NaN,NaN
1,28Aug22 ARP,1.0,"Mr Dash Fire (Flores, James)",124 BL b,1.0,1,1,1,0.90*,20.411,86.0,broke best edg
2,7 4,NaN,NaN,NaN,NaN,Neck,1/2,1/2,NaN,NaN,NaN,NaN
3,14Aug22 ARP,5.0,"Flare for Royalty (Lazalde, Martin)",124 BL bf,2.0,2,2,2,8.30,20.482,85.0,away well outfi
4,8 4,NaN,NaN,NaN,NaN,1 1/2,2,Head,NaN,NaN,NaN,NaN
5,24Jul22 ARP,8.0,"Yesterday Road (Aguirre-Erives, Cristian)",124 BL bf,7.0,5,5,3,2.90,20.569,82.0,broke out shar
6,2 1,NaN,NaN,NaN,NaN,Neck,Head,Neck,NaN,NaN,NaN,NaN
7,22Aug22 ARP,3.0,"Paint Your Face Blue (Garcia, Ramiro)",124 B bf,4.0,3,3,4,9.40,20.583,82.0,away well caug
8,1 3,NaN,NaN,NaN,NaN,Neck,Head,2 1/2,NaN,NaN,NaN,NaN
9,5Aug22 PRM,2.0,"Carisma Vista (Montalvo, Genovero)",124 BL b,3.0,4,4,5,5.30,20.633,81.0,away well outfi


In [8]:
#Testing for whole pdf
master_df = pd.DataFrame()
page_list = race_results_scanner.get_page_list(pdf)
print(page_list)
    
reader = PdfReader(pdf) #File to be scanned
number_of_pages = len(reader.pages) #Number of pages
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()
i = 0
for page in page_list:
    header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
    result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan
                
    #Combine into page DF
    top_table = result_tables[0]
    bottom_table = result_tables[1]
    #Dropping horse name for merge
    bottom_table = bottom_table.drop("Horse Name", axis = 1)
    merged_df = top_table.join(bottom_table.set_index("Pgm"), on = "Pgm", rsuffix = "_RLP")
    for field,value in header.items():
        merged_df[field] = value
        #Changing all cols to object type 
        merged_df = merged_df.astype(object)
        #For every page df
    #Merge into master pdf df
    if(len(pdf_df) < 1):
        pdf_df = merged_df
    else:
        pdf_df = pd.merge(pdf_df, merged_df, how = 'outer')
            
    print(pdf_df)
    #Merging into master 
    #if(len(master_df) < 1):
     #   master_df = pdf_df
    #else:
    #    master_df = pd.merge(master_df,pdf_df, how = 'outer')

The PDF <_io.BufferedReader name='Error_PDFS/ARP_10_01_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


[{'page_num': 0, 'horse_count': 6}, {'page_num': 1, 'horse_count': 7}, {'page_num': 2, 'horse_count': 6}, {'page_num': 3, 'horse_count': 8}, {'page_num': 4, 'horse_count': 6}, {'page_num': 5, 'horse_count': 5}, {'page_num': 6, 'horse_count': 6}, {'page_num': 7, 'horse_count': 6}, {'page_num': 8, 'horse_count': 9}]
Looking for tables...
Found table top bound at 125.53800000000001
Found table bottom bound at 206.83799999999997
Looking for tables...
Found table top bound at 358.89
Found table bottom bound at 443.52400000000006


KeyError: 'Start'

In [6]:
pdf_df.to_csv("ARAP.csv")